# A simple chatbot agent

## Prepare the environment

In [ ]:
import sys
sys.path.append('../../..')

from autogen import ConversableAgent, register_function

from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent
from inswitch.agent.apiagent import ApiAgent
from usecases.fill.filluc.mockupnerv.session import make_request

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## The simplied doc providing agent: no RAG in it - only manually selected text

In [2]:
with open("../data/nerve_api_dna.txt", "r") as api_doc:
    content = api_doc.read()

api_doc_provider = get_fixed_reply_agent(
    'api_doc_provider', 
    reply=content
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)


## Here are the API invocation agents

In [3]:
nerv_api_system_message = '''
You have access to a tool to call the Nerve DNA API to fulfil the user's intent.
The task is in the context.
You will get from the context a document of the Nerve DNA API.
Using the document, you will figure out how to call the API, i.e., using what endpoint and method.
You will also need to generate the configuration file as the data of the api call, to fulfil the task assigned to you.
Always use the concrete version numbers (digits), not the version names.
If you don't have the version for some workloads, 
you need to call the API to get the available version numbers and their hash code. Use the first version by default 

'''

nerve_api_agent = ApiAgent(
    "nerve_api_agent", 
    system_message=nerv_api_system_message, 
    max_internal_turns=6
)


nerve_api_agent.register_api_function(
    make_request,
    description = "This is the functionto send a request to the Nerve API for deploying workloads etc. "
)




The following parameters of the function 'make_request' with default values are not annotated: 'files', 'workaround'.


## Here we go!
A simple sequence: user_intent -> api_doc_provider -> nerv_tool_executor

Current intent is hard coded.

In [4]:


deploy_plan = """Deploy the following workloads for Machine: M00001 (Type: MTC)

      - Workload: ngix, Version: ngix_27 (1.27.2)
      - Workload: nodejs, Version: nodejs_23 (23.2.0)
      - Workload: alarm_record
      - Workload: mqtt_sender
      
"""


deployment_plan_announcer = get_fixed_reply_agent(
    'deployment_plan_announcer',
    reply=deploy_plan
)

chat_result = moderator.initiate_chats(
    [
        {
            "recipient": deployment_plan_announcer,
            "message": "What should be deployed? on which machine?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": api_doc_provider,
            "message": "Please provide me the api doc",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": nerve_api_agent,
            "message": "you can see the intent and the knowledge in the context",
            "max_turns": 1,
            "summary_method": "last_msg"
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to deployment_plan_announcer):

What should be deployed? on which machine?

--------------------------------------------------------------------------------
deployment_plan_announcer (to moderator):

Deploy the following workloads for Machine: M00001 (Type: MTC)

      - Workload: ngix, Version: ngix_27 (1.27.2)
      - Workload: nodejs, Version: nodejs_23 (23.2.0)
      - Workload: alarm_record
      - Workload: mqtt_sender
      


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to api_doc_provider):

Please provide me the api doc
Context: 
Deploy the following

## Next steps:

1. Replace current api_doc_provider (a fixed_reply_agent) by a RAG assistant, that extracts from a complete API document the parts that are relevant to DNA and deployment.
2. Add in the beginning of the sequence another agent (or agents) to extract the list of workloads. Eventually, the input intent should be like "I want to be able to monitor the thermal stability of machine M0001"